In [1]:
!pip install transformers

In [2]:
import transformers

In [3]:
!pip install streamlit

  Using cached streamlit-1.22.0-py2.py3-none-any.whl (8.9 MB)
  Using cached pydeck-0.8.1b0-py2.py3-none-any.whl (4.8 MB)
  Using cached validators-0.20.0.tar.gz (30 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 238.7/238.7 kB 1.3 MB/s eta 0:00:00
  Using cached GitPython-3.1.31-py3-none-any.whl (184 kB)
  Using cached tzlocal-4.3-py3-none-any.whl (20 kB)
  Using cached altair-4.2.2-py3-none-any.whl (813 kB)
     -------------------------------------- 904.0/904.0 kB 7.2 MB/s eta 0:00:00
  Using cached blinker-1.6.2-py3-none-any.whl (13 kB)
     ---------------------------------------- 21.5/21.5 MB 7.6 MB/s eta 0:00:00
  Using cached Pympler-1.0.1-py3-none-any.whl (164 kB)
  Using cached gitdb-4.0.10-py3-none-any.whl (62 kB)
     ---------------------------------------- 84.5/84.5 kB 4.6 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 9.1 MB/s eta 0:00:00
  Us

In [4]:
from  transformers import VisionEncoderDecoderModel , ViTFeatureExtractor, AutoTokenizer
import streamlit as st
from PIL import Image
import torch
import os



In [6]:
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

In [7]:
feature_extractor =ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0): ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, 

In [9]:
max_length=16
num_beams=4
gen_kwargs={"max_length":max_length , "num_beams":num_beams}

In [10]:
def predict_step(image_paths):
    images=[]
    for image_path in image_paths:
        i_image =Image.open(image_path)
        if i_image.mode !="RGB":
            i_image = i_image.convert(mode="RGB")
        images.append(i_image)
        
    pixel_values = feature_extractor(images=images,return_tensors='pt').pixel_values
    pixel_values=pixel_values.to(device)
    output_ids = model.generate(pixel_values, **gen_kwargs)
    preds=tokenizer.batch_decode(output_ids,skip_special_tokens=True)
    preds=[pred.strip() for pred in preds]
    return preds

In [17]:
predict_step(["D:\ML\ImageGenerator\Image2.png"])

['a woman is standing in a field with a horse']

In [13]:
def app():
    st.title("Image Captioning with Hugging Face")
    st.markdown("Upload an image to generate a caption.")
    
    # Upload image
    image = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])
    
    # Generate caption
    if image is not None:
        caption = predict_step(image)
        st.success(caption)

if __name__ == "__main__":
    app()


2023-05-06 12:56:36.419 
  command:

    streamlit run D:\Anaconda\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
